In [26]:
import pandas as pd
import numpy as nm
data=pd.read_csv('C:\\Users\\rohit\\OneDrive\\Desktop\\files\\16MIS0299\\code\\heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [37]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')
#feature_cols=['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal']
#X=data[feature_cols]
#y=data.target

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
age,1.000000,-0.098447,-0.068653,0.279351,0.213678,0.121308,-0.116211,-0.398522,0.096801,0.210013,-0.168814,0.276326,0.068001,-0.225439
sex,-0.098447,1.000000,-0.049353,-0.056769,-0.197912,0.045032,-0.058196,-0.044020,0.141664,0.096093,-0.030711,0.118261,0.210041,-0.280937
cp,-0.068653,-0.049353,1.000000,0.047608,-0.076904,0.094444,0.044421,0.295762,-0.394280,-0.149230,0.119717,-0.181053,-0.161736,0.433798
trestbps,0.279351,-0.056769,0.047608,1.000000,0.123174,0.177531,-0.114103,-0.046698,0.067616,0.193216,-0.121475,0.101389,0.062210,-0.144931
chol,0.213678,-0.197912,-0.076904,0.123174,1.000000,0.013294,-0.151040,-0.009940,0.067023,0.053952,-0.004038,0.070511,0.098803,-0.085239
fbs,0.121308,0.045032,0.094444,0.177531,0.013294,1.000000,-0.084189,-0.008567,0.025665,0.005747,-0.059894,0.137979,-0.032019,-0.028046
restecg,-0.116211,-0.058196,0.044421,-0.114103,-0.151040,-0.084189,1.000000,0.044123,-0.070733,-0.058770,0.093045,-0.072042,-0.011981,0.137230
thalach,-0.398522,-0.044020,0.295762,-0.046698,-0.009940,-0.008567,0.044123,1.000000,-0.378812,-0.344187,0.386784,-0.213177,-0.096439,0.421741
exang,0.096801,0.141664,-0.394280,0.067616,0.067023,0.025665,-0.070733,-0.378812,1.000000,0.288223,-0.257748,0.115739,0.206754,-0.436757
oldpeak,0.210013,0.096093,-0.149230,0.193216,0.053952,0.005747,-0.058770,-0.344187,0.288223,1.000000,-0.577537,0.222682,0.210244,-0.430696


In [38]:
cp = pd.get_dummies(data['cp'], prefix = "cp", drop_first=True)
thal = pd.get_dummies(data['thal'], prefix = "thal" , drop_first=True)
slope = pd.get_dummies(data['slope'], prefix = "slope", drop_first=True)
new_data = pd.concat([data, cp, thal, slope], axis=1)
new_data.drop(['cp', 'thal', 'slope'], axis=1, inplace=True)
new_data

,age,sex,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,ca,target,cp_1,cp_2,cp_3,thal_1,thal_2,thal_3,slope_1,slope_2
0,63,1,145,233,1,0,150,0,2.3,0,1,0,0,1,1,0,0,0,0
1,37,1,130,250,0,1,187,0,3.5,0,1,0,1,0,0,1,0,0,0
2,41,0,130,204,0,0,172,0,1.4,0,1,1,0,0,0,1,0,0,1
3,56,1,120,236,0,1,178,0,0.8,0,1,1,0,0,0,1,0,0,1
4,57,0,120,354,0,1,163,1,0.6,0,1,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,140,241,0,1,123,1,0.2,0,0,0,0,0,0,0,1,1,0
299,45,1,110,264,0,1,132,0,1.2,0,0,0,0,1,0,0,1,1,0
300,68,1,144,193,1,1,141,0,3.4,2,0,0,0,0,0,0,1,1,0
301,57,1,130,131,0,1,115,1,1.2,1,0,0,0,0,0,0,1,1,0


In [39]:
X = new_data.iloc[:,:-1].values
y = new_data.iloc[:,-1].values
print('shape of X: ',X.shape)

shape of X:  (303, 18)


In [40]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)

In [41]:
from sklearn.preprocessing import StandardScaler
Sc = StandardScaler()
X_train = Sc.fit_transform(X_train)
X_test = Sc.transform(X_test)

In [42]:
from xgboost import XGBClassifier
model=XGBClassifier()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)

[14:41:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\rohit\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [43]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[40,  2],
       [ 0, 34]], dtype=int64)

In [44]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

Accuracy: 0.9736842105263158
Precision: 0.9444444444444444
Recall: 1.0


In [ ]:
from sklearn.model_selection import GridSearchCV
XG = XGBClassifier(random_state=0)                           # Instantiate XG Boost Classifier model
param = {"n_estimators"     : [100, 200,300],
         "learning_rate"    : [0.1, 0.3, 0.5, 0.6],
         "max_depth"        : [2, 3, 4, 5],
         "min_child_weight" : [1, 2, 3, 4],
         "gamma"            : [0.5, 0.8, 0.9, 1],
         "colsample_bytree" : [0.7, 0.8],
         "subsample"        : [0.7, 0.8],
         }                                                        # Set parameters of the model          
grid_search = GridSearchCV(estimator = XG, param_grid = param, 
                          cv = 5, n_jobs = -1, verbose = 2)          # Instantiate Grid Search model
grid_search = grid_search.fit(X_train, y_train)                      # Train Gird Search model

print("best accuracy is :" , grid_search.best_score_)                # Display best accuracy

grid_search.best_params_    # best_parms_  is a method in Grid Search to return The Best with resepct to the Metric

Fitting 5 folds for each of 3072 candidates, totalling 15360 fits


In [ ]:

from sklearn.metrics import plot_confusion_matrix
XGB = XGBClassifier(n_estimators=100, gamma=0.8, learning_rate=0.1,
                      max_depth=5, min_child_weight=1, subsample=0.7,
                      colsample_bytree=0.7, random_state=0)                 # Instantiate XG Boost Classifier model
XGB.fit(X_train, y_train)                                                   # Train XG Boost Classifier model
print('train accuracy', XGB.score(X_train, y_train))                        # Display train accuracy
print('test accuracy', XGB.score(X_test, XGB.predict(X_test)))           # Display test accuracy


In [ ]:
#After correlation 
print("Before Grid Search Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("After Grid Search Accuracy: ",grid_search.best_score_)

In [ ]:
#Before correlation 
print("Before Grid Search Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("After Grid Search Accuracy: ",grid_search.best_score_)